In [47]:
from pyspark.sql.types import *
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import lit, current_date, col, when,window,to_date,row_number
from pyspark.sql.functions import lit, current_date, row_number, col, when, concat_ws, md5
from pyspark.sql.functions import date_format
from pyspark.sql import Window
import os
import subprocess
import hashlib
from pyspark.sql.functions import to_timestamp


In [86]:
#sc.stop()

In [97]:
spark = SparkSession\
    .builder\
    .master("local[4]")\
    .appName("branches")\
    .config("spark.eventLog.logBlockUpdates.enabled", True)\
    .getOrCreate()

sc = spark.sparkContext

In [68]:
spark.conf.set("spark.sql.repl.eagerEval.maxColWidth", 1000)

In [69]:

def get_lateset_file(directory):
    fs = spark._jvm.org.apache.hadoop.fs.FileSystem.get(spark._jsc.hadoopConfiguration())
    list_status = fs.listStatus(spark._jvm.org.apache.hadoop.fs.Path(directory))
    files = [(file.getPath().toString(), file.getModificationTime()) for file in list_status if file.isFile()]
    files.sort(key=lambda x: x[1], reverse=True)
    latest_file = files[0][0] if files else None
    return latest_file


In [99]:
latest_file = get_lateset_file('/user/branches')
print(latest_file)

hdfs://localhost:9000/user/branches/branches_SS_raw_2.csv


In [101]:
checkpoint_path = "/user/checkpoint/checkpoint_branches.txt"
result = subprocess.run(['hdfs', 'dfs', '-test', '-e', checkpoint_path])
if result.returncode != 0:
    latest_processed_file = ""
else:
    rdd = sc.textFile("/user/checkpoint/checkpoint_branches.txt")
    latest_processed_file = rdd.take(rdd.count())[-1]
    print(latest_processed_file)

hdfs://localhost:9000/user/branches/branches_SS_raw_2.csv


In [72]:
schema = StructType([
    StructField("branch_id", IntegerType(), True),
    StructField("location", StringType(), True),
    StructField("establish_date",StringType(), True),
    StructField("class", StringType(), True),
])

In [73]:
#latest_file = get_lateset_file('/user/branches')


In [74]:
#df_current = spark.read.csv(latest_file, header=True, schema=schema) 

In [75]:
#df = df_current.withColumn("establish_date", to_timestamp(col("establish_date"), "M/d/yyyy"))


In [76]:
#df.show()

In [77]:
schema_old = StructType([
    StructField("branch_id", IntegerType(), True),
    StructField("location", StringType(), True),
    StructField("establish_date", TimestampType(), True),
    StructField("class", StringType(), True),
    StructField("current_flag", BooleanType(), True),
    StructField("effective_date", TimestampType(), True),
    StructField("expiration_date", TimestampType(), True),
    StructField("sk_branch_id", TimestampType(), True)
])

In [78]:
def write_checkpoint(checkpoint_path, latest_file):
    checkpoint_dir = os.path.dirname(checkpoint_path)
    result = subprocess.run(['hdfs', 'dfs', '-test', '-e', checkpoint_dir])
    if result.returncode != 0:
        subprocess.run(['hdfs', 'dfs', '-mkdir', '-p', checkpoint_dir])
    
    with open('/tmp/checkpoint_tmp.txt', 'w') as f:
        f.write(latest_file)
    
    subprocess.run(['hdfs', 'dfs', '-put', '-f', '/tmp/checkpoint_tmp.txt', checkpoint_path])
    os.remove('/tmp/checkpoint_tmp.txt')

In [79]:
def column_renamer(df, suffix, append):
    if append:
        new_column_names = list(map(lambda x: x + suffix, df.columns))
    else:
        new_column_names = list(map(lambda x: x.replace(suffix, ""), df.columns))
    return df.toDF(*new_column_names)

In [107]:
df_current = spark.read.csv(latest_file, header=True, inferSchema=True) 
print(f'Data types of all the columns is : {df_current.dtypes}')
df_current = df_current.withColumn("establish_date", to_timestamp(col("establish_date"), "M/d/yyyy"))

Data types of all the columns is : [('branch_id', 'int'), ('location', 'string'), ('establish_date', 'timestamp'), ('class', 'string')]


In [108]:
df_current.show()

+---------+-----------+-------------------+-----+
|branch_id|   location|     establish_date|class|
+---------+-----------+-------------------+-----+
|        1|   New York|2017-01-15 00:00:00|    A|
|        2|Los Angeles|2016-07-28 00:00:00|    B|
|        3|    Chicago|2015-03-10 00:00:00|    A|
|        4|    Houston|2016-11-05 00:00:00|    D|
|        5|    Phoenix|2017-09-20 00:00:00|    C|
+---------+-----------+-------------------+-----+



In [80]:
def get_hash(df, keys_list):
    columns = [col(column) for column in keys_list]
    if columns:
        return df.withColumn("hash_md5", md5(concat_ws("", *columns)))
    else:
        return df.withColumn("hash_md5", md5(lit(1)))

In [113]:
processed_path = "/user/silver/branches/branch_dim.parquet"
scd_path = "/user/silver/branches/scd2"
processed_dir = "/user/silver/branches"
checkpoint_path = "/user/checkpoint/checkpoint_branches.txt"

EOW_DATE = "9999-12-31"
DATE_FORMAT = "yyyy-MM-dd"
type2_cols = ["location", "establish_date", "class"]
latest_file = get_lateset_file('/user/branches')

if latest_processed_file == latest_file:
    print(f"File already processed before: {latest_file}")
    spark.stop()
else:
    result = subprocess.run(['hdfs', 'dfs', '-test', '-e', processed_dir])
    if result.returncode == 0:
        
        df_current = spark.read.csv(latest_file, header=True, inferSchema=True) 
        df_current = df_current.withColumn("establish_date", to_timestamp(col("establish_date"), "M/d/yyyy"))
        df_current.show()
        print(f'Data types of all the columns is : {df_current.dtypes}')
        
        window_spec = Window.orderBy("branch_id")
        
        result = subprocess.run(['hdfs', 'dfs', '-test', '-e', scd_path])
        if result.returncode == 0:
            df_history = spark.read.parquet("/user/silver/branches/scd2/*.parquet")\
                        .withColumn("current_flag", lit(True)) \
                        .withColumn("sk_branch_id", row_number().over(window_spec))
        else:
            df_history = spark.read.parquet("/user/silver/branches/*.parquet")\
            .withColumn("current_flag", lit(True)) \
            .withColumn("sk_branch_id", row_number().over(window_spec))
        
        df_history.show()
        
        print(f'Data types of all the columns is : {df_history.dtypes}')
        
        df_history_open = df_history.where(col("current_flag") == True)
        df_history_closed = df_history.where(col("current_flag") == False)
        
        max_sk = df_history_open.agg({"sk_branch_id": "max"}).collect()[0][0]
        
        df_history_open_hash = column_renamer(get_hash(df_history_open, type2_cols), suffix="_history", append=True)
        df_current_hash = column_renamer(get_hash(df_current, type2_cols), suffix="_current", append=True)
        
        
                        
                        
        df_merged = df_history_open_hash \
            .join(df_current_hash, col("branch_id_current") == col("branch_id_history"), how="full_outer") \
            .withColumn("Action", 
                when(col("hash_md5_current") == col("hash_md5_history"), 'NOCHANGE')
                .when(col("branch_id_current").isNull(), 'DELETE')
                .when(col("branch_id_history").isNull(), 'INSERT')
                .when(col("branch_id_history") == col("branch_id_current"), 'NOCHANGE')  # Corrected this line
                .otherwise('UPDATE'))

        
        df_merged.show()
        
        
        window_spec  = Window.orderBy("branch_id")


        df_nochange = column_renamer(df_merged.filter(col("action") == 'NOCHANGE'), suffix="_history", append=False)\
                        .select(df_history_open.columns)

        df_nochange.show()
        
        df_insert = column_renamer(df_merged.filter(col("action") == 'INSERT'), suffix="_current", append=False)\
                .select(df_current.columns)\
                .withColumn("effective_date",date_format(current_date(),DATE_FORMAT))\
                .withColumn("expiration_date",date_format(lit(EOW_DATE),DATE_FORMAT))\
                .withColumn("row_number",row_number().over(window_spec))\
                .withColumn("sk_branch_id",col("row_number")+ max_sk)\
                .withColumn("current_flag", lit(True))\
                .drop("row_number")

        df_insert.show()
        
        max_sk_i = df_insert.agg({"sk_branch_id": "max"}).collect()[0][0]

        df_deleted = column_renamer(df_merged.filter(col("action") == 'DELETE'), suffix="_history", append=False)\
                .select(df_history_open.columns)\
                .withColumn("expiration_date", date_format(current_date(),DATE_FORMAT))\
                .withColumn("current_flag", lit(False))

        df_deleted.show()
        
        
        df_update = column_renamer(df_merged.filter(col("action") == 'UPDATE'), suffix="_history", append=False)\
                .select(df_history_open.columns)\
                .withColumn("expiration_date", date_format(current_date(),DATE_FORMAT))\
                .withColumn("current_flag", lit(False))\
            .unionByName(
            column_renamer(df_merged.filter(col("action") == 'UPDATE'), suffix="_current", append=False)\
                .select(df_current.columns)\
                .withColumn("effective_date",date_format(current_date(),DATE_FORMAT))\
                .withColumn("expiration_date",date_format(lit(EOW_DATE),DATE_FORMAT))\
                .withColumn("row_number",row_number().over(window_spec))\
                .withColumn("sk_branch_id",col("row_number")+ max_sk_i)\
                .withColumn("current_flag", lit(True))\
                .drop("row_number")
                )
        df_update.show()
        
        df_final = df_history_closed\
            .unionByName(df_nochange)\
            .unionByName(df_insert)\
            .unionByName(df_deleted)\
            .unionByName(df_update)
        df_final.show()
        #df_final.write.parquet(scd_path)
        #write_checkpoint(checkpoint_path, latest_file)
    else:
        df_new = spark.read.csv(latest_file, header=True, inferSchema=True)\
            .withColumn("current_flag", lit(True)) \
            .withColumn("effective_date", current_date()) \
            .withColumn("expiration_date", lit(EOW_DATE).cast("date"))
        
        write_checkpoint(checkpoint_path, latest_file)
        print(f"New file processed: {latest_file}")
        df_new.write.parquet(processed_path)

+---------+-----------+-------------------+-----+
|branch_id|   location|     establish_date|class|
+---------+-----------+-------------------+-----+
|        1|   New York|2017-01-15 00:00:00|    A|
|        2|Los Angeles|2016-07-28 00:00:00|    B|
|        3|    Chicago|2015-03-10 00:00:00|    A|
|        4|    Houston|2016-11-05 00:00:00|    D|
|        5|    Phoenix|2017-09-20 00:00:00|    C|
+---------+-----------+-------------------+-----+

Data types of all the columns is : [('branch_id', 'int'), ('location', 'string'), ('establish_date', 'timestamp'), ('class', 'string')]
+---------+-----------+-------------------+-----+------------+--------------+---------------+------------+
|branch_id|   location|     establish_date|class|current_flag|effective_date|expiration_date|sk_branch_id|
+---------+-----------+-------------------+-----+------------+--------------+---------------+------------+
|        1|   New York|2017-01-15 00:00:00|    A|        true|    2024-07-12|     2024-07-12

In [84]:
df_final = spark.read.parquet("/user/silver/branches/branch_dim.parquet")


In [85]:
df_final.show()

+---------+-----------+-------------------+-----+------------+--------------+---------------+------------+
|branch_id|   location|     establish_date|class|current_flag|effective_date|expiration_date|sk_branch_id|
+---------+-----------+-------------------+-----+------------+--------------+---------------+------------+
|        2|Los Angeles|2016-07-28 00:00:00|    B|        true|    2024-07-12|     9999-12-31|           2|
|        1|   New York|2017-01-15 00:00:00|    A|       false|    2024-07-12|     2024-07-12|           1|
|        3|    Chicago|2015-03-10 00:00:00|    A|        true|    2024-07-12|     9999-12-31|           3|
|        5|    Phoenix|2017-09-20 00:00:00|    C|        true|    2024-07-12|     9999-12-31|           5|
|        4|    Houston|2016-11-05 00:00:00|    D|        true|    2024-07-12|     9999-12-31|           4|
|        1|      tanta|2017-01-15 00:00:00|    A|        true|    2024-07-12|     9999-12-31|           7|
|        6|       Alex|2017-09-21 00:

In [52]:
df_current = spark.read.csv(latest_file, header=True, schema=schema) 

In [53]:
print(f'Data types of all the columns is : {df_current.dtypes}')

Data types of all the columns is : [('branch_id', 'int'), ('location', 'string'), ('establish_date', 'string'), ('class', 'string')]


In [51]:
print(f'Data types of all the columns is : {df_new.dtypes}')

Data types of all the columns is : [('branch_id', 'int'), ('location', 'string'), ('establish_date', 'timestamp'), ('class', 'string'), ('current_flag', 'boolean'), ('effective_date', 'date'), ('expiration_date', 'date')]


In [14]:
df_new.show()

+---------+-----------+-------------------+-----+----------+-------------------+-------------------+
|branch_id|   location|     establish_date|class|is_current|         start_date|           end_date|
+---------+-----------+-------------------+-----+----------+-------------------+-------------------+
|        1|   New York|2017-01-15 00:00:00|    A|      true|2024-07-11 00:00:00|9999-12-31 00:00:00|
|        2|Los Angeles|2016-07-28 00:00:00|    B|      true|2024-07-11 00:00:00|9999-12-31 00:00:00|
|        3|    Chicago|2015-03-10 00:00:00|    A|      true|2024-07-11 00:00:00|9999-12-31 00:00:00|
|        4|    Houston|2016-11-05 00:00:00|    D|      true|2024-07-11 00:00:00|9999-12-31 00:00:00|
|        5|    Phoenix|2017-09-20 00:00:00|    C|      true|2024-07-11 00:00:00|9999-12-31 00:00:00|
+---------+-----------+-------------------+-----+----------+-------------------+-------------------+



In [16]:
print(f'Data types of all the columns is : {df_final.dtypes}')

Data types of all the columns is : [('branch_id', 'int'), ('location', 'string'), ('establish_date', 'date'), ('class', 'string'), ('is_current', 'boolean'), ('start_date', 'date'), ('end_date', 'date'), ('sk_branch_id', 'int')]


In [171]:
df_existing.show()

+---+-----------+------------------------+---+----+-------------------+-------------------+
|  1|   New York|2017-01-15T00:00:00.000Z|  A|true|         2024-07-11|         9999-12-31|
+---+-----------+------------------------+---+----+-------------------+-------------------+
|  2|Los Angeles|     2016-07-28 00:00:00|  B|true|2024-07-11 00:00:00|9999-12-31 00:00:00|
|  3|    Chicago|     2015-03-10 00:00:00|  A|true|2024-07-11 00:00:00|9999-12-31 00:00:00|
|  4|    Houston|     2016-11-05 00:00:00|  D|true|2024-07-11 00:00:00|9999-12-31 00:00:00|
|  5|    Phoenix|     2017-09-20 00:00:00|  C|true|2024-07-11 00:00:00|9999-12-31 00:00:00|
+---+-----------+------------------------+---+----+-------------------+-------------------+



In [95]:
sc.stop()